<a href="https://colab.research.google.com/github/SojeongShin/NLP_2023/blob/Notion/%EB%8B%A8%EC%96%B4_%EC%9E%84%EB%B2%A0%EB%94%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [자연어처리]
# 11주차(11-1). 단어 임베딩

# 202001581 신소정 <- 본인의 학번, 이름으로 변경하여 제출
* **모든 셀 실행** 후 제출하시기 바랍니다.
* **실습 (11-1-1, 11-1-2)**이 있습니다. (제출 기한: 11/14(화) 23시 59분까지)

# 이제 Colab 사용에 모든 수강생분들이 어느정도 익숙해 졌을 거라 생각됩니다!
> ~~ ##### 중간 중간 **'### ... 이 부분을 완성하시오'** 라는 부분의 코드를 완성해야 합니다. ~~ (이번 시간은 패스)





# 단어 임베딩 학습하기

## [지난 시간 Review 시작]

## 영화 리뷰 데이터 셋 다운로드

In [1]:
!wget -nc https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt

--2023-11-14 04:23:51--  https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19515078 (19M) [text/plain]
Saving to: ‘ratings.txt’

ratings.txt         100%[===================>]  18.61M   119MB/s    in 0.2s    

2023-11-14 04:23:52 (119 MB/s) - ‘ratings.txt’ saved [19515078/19515078]



## 필요한 모듈 준비
* Word2Vec
* konlpy
* time

In [2]:
! pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 66.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 51.7 MB/s eta 0:00:00


In [3]:
from gensim.models import Word2Vec
from konlpy.tag import Komoran
import time

## 파일 읽기 함수 작성

In [4]:
def read_review(filename):
    f = open(filename, 'r', encoding = 'utf-8')
    data = []
    for line in f.readlines():
        if line != '':
            data.append(line.split('\t'))
    data = data[1:]
    f.close()

    return data

In [5]:
data = read_review('ratings.txt')[:20000]
len(data)

20000

## 형태소 분석

In [6]:
# 1분 내외 소요
print('POS tagging...')
start = time.time()
komoran = Komoran()
docs = []
count = 0
for sent in data:
    if count % 1000 == 0:
        print(count, time.time()-start)
    docs.append(komoran.morphs(sent[1]))
    count += 1
print(docs[:10])
print('Done...', time.time()-start)

POS tagging...
0 7.2439939975738525
1000 8.92544960975647
2000 11.128210067749023
3000 15.194361209869385
4000 16.50486660003662
5000 18.801929235458374
6000 19.42146873474121
7000 20.052690029144287
8000 20.738078594207764
9000 21.24489164352417
10000 21.90714406967163
11000 22.62558674812317
12000 23.16317629814148
13000 23.80304265022278
14000 24.32591485977173
15000 24.96351170539856
16000 25.60774278640747
17000 26.16112971305847
18000 26.914756059646606
19000 27.57035231590271
[['어리', 'ㄹ', '때', '보고', '지금', '다시', '보', '아도', '재밌어요ㅋㅋ'], ['디자인', '을', '배우', '는', '학생', '으로', ',', '외국', '디자이너', '와', '그', '들', '이', '일구', 'ㄴ', '전통', '을', '통하', '아', '발전', '하', '아', '가', '는', '문화', '산업', '이', '부럽', '었', '는데', '.', '사실', '우리나라', '에서', '도', '그', '어렵', 'ㄴ', '시절', '에', '끝', '까지', '열정', '을', '지키', 'ㄴ', '노라', '노', '같', '은', '전통', '이', '있', '어', '저', '와', '같', '은', '사람', '들', '이', '꿈', '을', '꾸', '고', '이루', '어', '나가', 'ㄹ', '수', '있', '다는', '것', '에', '감사', '하', 'ㅂ니다', '.'], ['폴리스', '스토리', '시리즈', '는',

## Word2Vec 학습

In [7]:
# 30초 내외 소요
print('Word2vec training...')
start = time.time()
model = Word2Vec(sentences=docs, vector_size=200, window=4, hs=1, min_count=5, sg=1, negative=0)
print('Done...', time.time()-start)

Word2vec training...
Done... 8.366213083267212


## 학습된 벡터 값 확인: Word2Vec

In [8]:
print('영화 :', model.wv['영화'])

영화 : [ 0.27476987  0.10328153  0.07004236 -0.16364959  0.00555059 -0.06527276
 -0.20747852  0.01050633  0.00809115  0.1437488   0.01618244 -0.0875892
  0.12016254 -0.01365201 -0.06589717  0.09441591  0.26640043 -0.0180727
  0.02379528 -0.20347653  0.10320098  0.11139355  0.07151501 -0.04259494
 -0.12508804  0.07576071 -0.03791439  0.09059676  0.07610602 -0.11089592
  0.03216196  0.1550722   0.00206434 -0.01631719  0.05008067  0.10491275
  0.0390149  -0.03224954 -0.07085147 -0.1379562  -0.07146536 -0.24406599
 -0.12444003 -0.01949303 -0.03317143 -0.19678327 -0.13711628 -0.22476766
  0.05328529 -0.01190276  0.00208987  0.03436305 -0.05262779  0.249729
  0.04341438  0.07751334  0.11587421 -0.00965243  0.013765   -0.07173312
 -0.1197695  -0.07747161  0.03414769  0.04430674 -0.17876881 -0.05122538
 -0.16519944 -0.14505802  0.05312873  0.13692448 -0.07497254  0.03240755
  0.08924672  0.07805929  0.02085966  0.14178866  0.34828475  0.00713791
 -0.08609683 -0.10161931  0.18080007  0.02804597  

## 단어 유사도 계산: Word2Vec

In [9]:
print(model.wv.most_similar('배우', topn=5))

[('조연', 0.6839469075202942), ('연기자', 0.6737328171730042), ('아역', 0.612555205821991), ('성우', 0.593385636806488), ('일품', 0.5877731442451477)]


In [10]:
print(model.wv.most_similar('이정재', topn=5))

[('전우치', 0.6738149523735046), ('쩔어', 0.6703853011131287), ('피트', 0.6633121967315674), ('재밋다', 0.6593860983848572), ('미남', 0.6466773748397827)]


In [11]:
print(model.wv.most_similar('감동', topn=5))

[('충격', 0.5810792446136475), ('낭만', 0.5458537340164185), ('쏠쏠', 0.47535523772239685), ('긍정', 0.4324502944946289), ('진하', 0.4212583005428314)]


## [지난 시간 Review 끝]

# fastText

## fastText 학습

In [12]:
# 30초 내외 소요
from gensim.models import FastText

print('FastText training...')
start = time.time()
model_fasttext = FastText(sentences=docs, vector_size=200, window=4, hs=1, min_count=5, sg=1, negative=0)
print('Done...', time.time()-start)

FastText training...
Done... 11.055439472198486


## 단어 유사도 계산: fastText

In [13]:
print(model_fasttext.wv.most_similar('배우', topn=5))

[('연기자', 0.6839979290962219), ('조연', 0.6819807887077332), ('연기파', 0.6736855506896973), ('여배우', 0.6342229843139648), ('성우', 0.6128546595573425)]


In [14]:
print(model_fasttext.wv.most_similar('이정재', topn=5))

[('김동욱', 0.6836695671081543), ('ㄷㄷㄷ', 0.665404200553894), ('이병헌', 0.6643837094306946), ('박보영', 0.6630793809890747), ('재밋네요', 0.6462324857711792)]


In [16]:
print(model_fasttext.wv.most_similar('감동', topn=5))

[('충격', 0.5420515537261963), ('낭만', 0.5383468270301819), ('쏠쏠', 0.45165807008743286), ('매혹', 0.4412870407104492), ('밀려오', 0.42429107427597046)]


---

# [[실습 11-1-1]] Word2Vec vs fastText: 결과 비교해보기

## 아래의 코드를 이용하여, 학습된 단어 집합(vocabulary)을 확인할 수 있다.

```
words = list(model.wv.index_to_key)
print(len(words))
print(words)
```
## 단어 집합 내의 다양한 측면의 임의의 단어를 선택해보고, 단어 임베딩별 결과를 비교 분석해 보시오.
</br>

## [결과 분석 방법]
* 단어 1: 디즈니
    * 단어 1에 대한 Word2Vec의 결과: [('고어', 0.6683346033096313), </br>('국산', 0.6509584784507751), ('왕조현', 0.650486171245575), </br>('전문', 0.6390817165374756), ('호러', 0.6338460445404053)]

    * 단어 1에 대한 fastText의 결과: [('공주', 0.5983179807662964), </br>('패스', 0.5938071608543396), ('페넬로페', 0.5881498456001282), </br>('대표작', 0.5874836444854736), ('역겹', 0.5864766836166382)]

    * 결과 비교 설명: '디즈니'에 대한 유사단어로는 '어린이', '공주', '픽사'와 같은 단어들을 예상했다. Word2Vec의 결과로는 거의 연관이 없는 단어들이 등장하였고, fastText는 그래도 '공주'와 같은 유사단어가 등장한 모습이다.
* 단어 2: 장르
    * 단어 2에 대한 Word2Vec의 결과: [('코미디', 0.5854723453521729), </br>('스케일', 0.566277027130127), ('스릴러', 0.5587427020072937), </br>('SF', 0.5526050925254822), ('요즘', 0.5402215719223022)]

    * 단어 2에 대한 fastText의 결과: [('스릴러', 0.6061246991157532), </br>('로맨스', 0.5781083106994629), ('SF', 0.5766094923019409), </br>('코미디', 0.5717133283615112), ('진수', 0.5615760684013367)]

    * 결과 비교 설명: '장르'에 대한 유사단어로는 '호러', '로맨스', '코미디'와 같은 단어들을 예상했다. Word2Vec와 fastText 모두 영화의 장르에 관련된 단어들을 출력했다. 그래도 fastText가 조금 더 유사단어가 잘 출력된 모습이다.
* 단어 3: 훈훈
    * 단어 3에 대한 Word2Vec의 결과: </br>
    [('뭉클', 0.6194627285003662), ('따뜻', 0.5887528657913208), </br>('찡하', 0.5675879716873169), ('따듯', 0.5337564945220947), </br>
    ('시리', 0.5309358835220337)]

    * 단어 3에 대한 fastText의 결과: </br>
    [('뭉클', 0.5952649116516113), ('찡하', 0.577792227268219), </br>
    ('따뜻', 0.5739107728004456), ('따듯', 0.5428399443626404), </br>
    ('먹먹', 0.5379146933555603)]

    * 결과 비교 설명: Word2Vec에는 '훈훈하다'라는 단어가 vocabulary set에 존재하지 않아 '훈훈'이라는 단어로 대체하여 결과를 보았다.'훈훈'에 대한 유사단어로는 '감동', '먹먹', '따뜻'과 같은 단어들을 예상했다. 두 방법 모두 연관 단어를 출력하였다.
    * 단어 1, 단어 2, 단어 3 비교를 통한 총평: Word2Vec은 단어를 최소단위로 쪼개어질 수 없는 단위로 판단하기에 말뭉치에 존재하지 않는 단어는 학습하지 못 한다. 반면에 내부 단어를 쪼개어 고려하는 fastText는 Word2Vec와 달리 말뭉치에 단어가 존재하지 않아도 내부 단어 분석으로 보다 정확한 결과를 도출할 수 있다. Word2Vec은 표준어가 대부분인 dataset에 대해 분석하면 정확도가 올라갈 것이다. fastText의 방법은 오타나 신조어가 있는 댓글과 같은 dataset에서 정확도가 올라갈 것이다.
</br>
</br>

## [가능한 경우에만 시도]
* 위의 총평에서 제안한 개선 방법을 시도해보고, 개선이 되었는지 혹은 예상과 달리 개선이 되지 않았는지 확인해보기

In [17]:
words = list(model.wv.index_to_key)
print(len(words))
print(words)

3817
['.', '이', '하', 'ㄴ', '는', '영화', '보', '다', '고', '의', '을', '았', '에', '게', '은', '가', '었', '도', 'ㄹ', '어', ',', '있', '들', '지', '...', '아', '!', '~', '를', '좋', 'ㅁ', '되', '재밌', '나', '정말', '는데', '어요', '최고', '너무', '적', '주', '?', '^', '네요', '것', '없', '지만', '잘', '수', '만', 'ㅂ니다', '과', '으로', '에서', '음', '습니다', 'ㄴ다', '같', '내', '않', '진짜', '연기', '던', '기', '감동', '..', '!!', '아서', '생각', '때', '로', '말', '점', '재미있', '그', '와', '안', '면', '거', '네', '시', '아니', '만들', '평점', '더', '드라마', '사람', '다시', '한', '재', '나오', '싶', '아도', '보다', '배우', '이런', '왜', '라', '사랑', '까지', '중', '겠', '번', '마지막', '10', '알', '어서', '스토리', '"', '오', '작품', '면서', '!!!', '요', ';', '명작', '짱', '남', '지금', '듯', '느끼', '많', 'ㄴ데', '1', '재미', '♥', '좀', '모르', '보이', '장면', '기대', '아요', '완전', '굿', '아름답', '감독', '뭐', '꼭', '또', '살', '역시', '이렇', '에게', '마음', '이것', '아야', '2', '내용', '으면', '씨', '액션', '다는', '라고', '대하', '좋아하', "'", '괜찮', '그리고', '라는', '넘', '시간', 'ㄴ가', '못하', '님', '기억', '많이', '낮', '가슴', '어리', '년', '인생', '이야기', '매력', 'ㅋㅋ', '그냥', '처음', '슬프', '스럽', '참', 

In [32]:
print(model_fasttext.wv.most_similar('훈훈하다', topn=5))

[('훈훈', 0.9967100620269775), ('뭉클', 0.5939434170722961), ('찡하', 0.5746933221817017), ('따뜻', 0.5731180310249329), ('따듯', 0.5433154106140137)]


In [34]:
print(model.wv.most_similar('훈훈', topn=5))

[('뭉클', 0.6194627285003662), ('따뜻', 0.5887528657913208), ('찡하', 0.5675879716873169), ('따듯', 0.5337564945220947), ('시리', 0.5309358835220337)]


---

# 사전 학습 단어 임베딩

---



In [ ]:
# ! pip install --upgrade gensim

In [35]:
# 1분 30초 내외 소요
# 미리 학습된 gensim data를 다운로드
import gensim.downloader as api

wv = api.load('glove-wiki-gigaword-50')

# 다른 사전 훈련 단어 임베딩을 사용해보고 싶다면,
# wv_word2vec = api.load('word2vec-google-news-300')
# wv_glove = api.load('glove-wiki-gigaword-300')
# wv_fasttext = api.load('fasttext-wiki-news-subwords-300')

[==================================================] 100.0% 66.0/66.0MB downloaded


In [36]:
type(wv)

gensim.models.keyedvectors.KeyedVectors

## 사전 학습된 벡터 값 확인: GloVe

In [37]:
vec_king = wv['king']
print('#Size of the vector:', len(vec_king))
print('#Vector for king:', vec_king)

#Size of the vector: 50
#Vector for king: [ 0.50451   0.68607  -0.59517  -0.022801  0.60046  -0.13498  -0.08813
  0.47377  -0.61798  -0.31012  -0.076666  1.493    -0.034189 -0.98173
  0.68229   0.81722  -0.51874  -0.31503  -0.55809   0.66421   0.1961
 -0.13495  -0.11476  -0.30344   0.41177  -2.223    -1.0756   -1.0783
 -0.34354   0.33505   1.9927   -0.04234  -0.64319   0.71125   0.49159
  0.16754   0.34344  -0.25663  -0.8523    0.1661    0.40102   1.1685
 -1.0137   -0.21585  -0.15155   0.78321  -0.91241  -1.6106   -0.64426
 -0.51042 ]


## 두 단어 간의 유사도 계산

In [38]:
print(wv.similarity('king', 'man'), 'vs', wv.similarity('king', 'woman'))

0.53093773 vs 0.41133785


In [39]:
print(wv.similarity('queen', 'man'), 'vs', wv.similarity('queen', 'woman'))

0.53667 vs 0.60031056


## 단어 유사도 계산: 사전 학습 GloVe

In [40]:
print('미니밴에 가까운 차:', wv.most_similar(positive=['car', 'minivan'], topn=3))

미니밴에 가까운 차: [('truck', 0.9100273251533508), ('suv', 0.904007613658905), ('jeep', 0.8619828820228577)]


In [41]:
print('여성, 왕에는 가까우면서 남성과는 먼 단어:',
      wv.most_similar(positive=['woman', 'king'], negative=['man'], topn=2))

여성, 왕에는 가까우면서 남성과는 먼 단어: [('queen', 0.8523604273796082), ('throne', 0.7664334177970886)]


In [42]:
print('breakfast cereal dinner lunch 중에서 다른 단어들과의 거리가 가장 먼 단어:',
      wv.doesnt_match("breakfast cereal dinner lunch".split()))

breakfast cereal dinner lunch 중에서 다른 단어들과의 거리가 가장 먼 단어: cereal


## 단어 집합 유사도 계산: 사전 학습 GloVe

In [43]:
print("distance between cat and dog: {:.2f}".format(wv.distance("cat", "dog")))
print("{:.4f}".format(wv.n_similarity(['bulgogi', 'shop'], ['japanese', 'restaurant'])))
print("{:.4f}".format(wv.n_similarity(['bulgogi', 'shop'], ['korean', 'restaurant'])))
print("{:.4f}".format(wv.n_similarity(['bulgogi', 'shop'], ['french', 'restaurant'])))

distance between cat and dog: 0.08
0.5375
0.5627
0.4377


# [[실습 11-1-2]] Word Analogy 다른 예시 생각해보기
## 단어를 임베딩 벡터로 만들면 의미적으로 유사한 단어들이 유사한 공간에 위치하게 된다.

## woman + king - man = ? 과 같이 또 어떤 예시가 있을까?


```
print('여성, 왕에는 가까우면서 남성과는 먼 단어:',
      wv.most_similar(positive=['woman', 'king'], negative=['man'], topn=1))
```
</br>

## [Word Analogy란?]
word analogy가 무엇인지 설명하시오.
* 설명: 임베딩을 평가하는 방법을 단어 유추 평가(word analogy test)라고 부릅니다.임베딩은 벡터인 만큼 사칙 연산이 가능합니다.
단어 벡터 간 덧셈/뺄셈을 통해 단어들 사이의 의미적, 문법적 관계를 도출해낼 수 있습니다.
 예를들면, 아들 - 딸 + 소녀 = 소년이 성립하면 성공적인 딘어유추라고 볼 수 있는데요. 아들 - 딸 사이의 관계와 소년 - 소녀 사이의 의미 차이가 임베딩에 함축돼 있으면 품질이 좋은 단어유추라 말할 수 있다는 이야기 입니다.

1. Paraphrase 정의, 주변 단어 분포들의 KL-Divergence (Paraphrase Error)가 작으면 Paraphrase 관계
2. PMI Matrix Column간의 연산을 수리적으로 정리하면 Column + Paraphrase Error + Conditional Independence Error + Independence Error로 표현 가능
3. PMI Matrix Column을 저차원으로 사영하면 Word Vector간 연산으로 표현 가능
4. 따라서 Paraphrase 관계이면 -> Word Vector간 연산으로 표현 가능 (Dependency Error Term 포함)
5. Paraphrase관계인 두 단어 집합에서 단어를 더하고 뺄 수 있음
6. 단어를 빼고 더해서 다른 단어로 변환이 가능하면 Word Transformation 성립, 이 때, 빼고 더한 단어를 Transformation Parameter로 정의
7. 특정 단어 쌍 집합에 속하는 모든 단어 쌍을 동시에 Transformation하는 6. Transformation Parameter가 존재한다면, Analogy 관계가 성립
8. 이 때, Analogy, Word Transformation, Paraphrase는 모두 필요충분조건
9. 결론: Analogy 관계이면 -> Word Vector간 연산으로 표현 가능 (Dependency Error Term 포함)

* 참고 자료 출처:
https://velog.io/@glad415/%EC%9E%84%EB%B2%A0%EB%94%A9Embedding%EC%9D%B4%EB%9E%80
 http://dsba.korea.ac.kr/seminar/?mod=document&uid=1421
</br>
</br>

## [예시 2개 이상 만들어보기]
word analogy를 확인해 볼 수 있는 응용 예시 2개 이상 만들어보기.
* 예시 1: girl + son - daughter = ?
    * 내가 예상한 결과: boy
    * 사전 학습된 GloVe의 결과: 예시 1: [('boy', 0.936298668384552)]

* 예시 2: woman + dad - man = ?
    * 내가 예상한 결과: mom
    * 사전 학습된 GloVe의 결과: 예시 2: [('mom', 0.9198637008666992)]


In [46]:
print('예시 2:',
      wv.most_similar(positive=['woman', 'dad'], negative=['man'], topn=1))

예시 2: [('mom', 0.9198637008666992)]


# [파일] -> [다운로드] -> [.ipynb 다운로드]
# 제출 후 **구글 계정 로그아웃** 잘 하시기 바랍니다.